<a href="https://colab.research.google.com/github/oliviasteeed/ChefGPT/blob/main/trying_LLM_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# olivia attempt llm fine tuning

In [2]:
!pip install datasets

In [4]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

In [4]:
# randomly sample 1000 rows from recipes dataset with like a million rows

import pandas as pd

# Define the path to your CSV file
csv_file = '/Users/oliviasteed/Desktop/full_dataset.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file)

# Randomly sample 1000 rows
sampled_df = df.sample(n=1000, random_state=42)  # random_state for reproducibility

# Optionally, inspect the first few rows of the sampled data
print(sampled_df.head())

# Save the sampled rows to a new CSV file (optional)
sampled_df.to_csv('/Users/oliviasteed/Desktop/1000_sample_dataset.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/oliviasteed/Desktop/full_dataset.csv'

In [7]:
# IMPORT DATA

# If your data is in a CSV file:
# from datasets import Dataset

# Replace this with the path to your dataset
data = Dataset.from_csv("/content/1000_sample_dataset.csv") #has recipes

In [4]:
data

Dataset({
    features: ['Unnamed: 0', 'title', 'ingredients', 'directions', 'link', 'source', 'NER'],
    num_rows: 1000
})

In [8]:
# Assuming 'column_name1' and 'column_name2' are the columns you want to remove
data = data.remove_columns(['link', 'source', 'Unnamed: 0'])

In [9]:
# SPLIT DATASET INTO TEST AND TRAIN

# Split dataset into training and validation sets (80% for training, 20% for validation)
train_test_split = data.train_test_split(test_size=0.2)

# Create training and validation datasets
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

In [10]:
# PREPROCESS THE DATA

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the entire dataset
def tokenize_function(examples):

  text = [f"{title} {ingredients} {directions}" for title, ingredients, directions in zip(examples["title"], examples["ingredients"], examples["directions"])]

    # Tokenize the concatenated text
  tokenized = tokenizer(text, padding="max_length", truncation=True)

     # Create labels (shifted input IDs)
  tokenized["labels"] = tokenized["input_ids"].copy()

  return tokenized

    # return tokenizer(examples["title", "ingredients", "directions"], padding="256", truncation=True)

tokenized_data = train_dataset.map(tokenize_function, batched=True) # Tokenize training data
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True) # Tokenize validation data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [9]:
# save tokenized data splits to disk

# Save the tokenized training dataset
tokenized_data.save_to_disk("/Users/oliviasteed/Desktop/chefgpt/traindata")

# Save the tokenized validation dataset
tokenized_val_dataset.save_to_disk("/Users/oliviasteed/Desktop/chefgpt/valdata")

Saving the dataset (0/1 shards):   0%|          | 0/800 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

In [11]:
# LOAD GPT MODEL

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Resize if adding custom tokens

Embedding(50257, 768)

In [12]:
# SET UP TRAINING ARGUMENTS

training_args = TrainingArguments(
    output_dir="/Users/oliviasteed/Desktop/chefgpt/gpt2_recipe_model",  # Directory to save the model
    overwrite_output_dir=True,
    num_train_epochs=6,  # Adjust the number of epochs
    per_device_train_batch_size=4,  # Adjust the batch size based on your GPU
    logging_dir="/Users.oliviasteed/Desktop/chefgpt/logs",
    logging_steps=500,
    save_steps=500,
    save_total_limit=2,
    eval_steps=500,  # Evaluate on validation set every 500 steps
    eval_strategy="steps",  # Evaluate on validation set during training
    learning_rate=5e-5,
    weight_decay=0.01,
    gradient_accumulation_steps=2,
)

In [ ]:
# TRAIN MODEL

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = tokenized_data,
    eval_dataset = tokenized_val_dataset,
    tokenizer = tokenizer,
)

trainer.train()

<ipython-input-13-a5a079c04c47>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: viaceee (viaceee-simon-fraser-university). Use `wandb login --relogin` to force relogin


In [2]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [30]:
# SAVE FINE TUNED MODEL

model.save_pretrained("/Content/ChefGPT/gpt2_recipe_model3")
tokenizer.save_pretrained("/Content/ChefGPT/gpt2_recipe_tokenizer3")

('/Content/ChefGPT/gpt2_recipe_tokenizer3/tokenizer_config.json',
 '/Content/ChefGPT/gpt2_recipe_tokenizer3/special_tokens_map.json',
 '/Content/ChefGPT/gpt2_recipe_tokenizer3/vocab.json',
 '/Content/ChefGPT/gpt2_recipe_tokenizer3/merges.txt',
 '/Content/ChefGPT/gpt2_recipe_tokenizer3/added_tokens.json')

In [31]:
# GENERATE RECIPES WITH MODEl

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained("/Content/ChefGPT/gpt2_recipe_model3")
tokenizer = GPT2Tokenizer.from_pretrained("/Content/ChefGPT/gpt2_recipe_tokenizer3")

# Function to generate a recipe based on ingredients
def generate_recipe(ingredients):
    prompt = f"Here's what you can make with: {ingredients} as key ingredients \n \nIngredients: {ingredients}\nInstructions:"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=300, num_return_sequences=1, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# try it out
ingredients = "rice, pineapple, chicken thighs"
recipe = generate_recipe(ingredients)
print("\n*******")
print(recipe)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



*******
Here's what you can make with: rice, pineapple, chicken thighs as key ingredients 
 
Ingredients: rice, pineapple, chicken thighs
Instructions: Put rice in a bowl and mix together. Add pineapple and chicken. Mix well. Serve hot or cold."]
